<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/Visualizacion_datos/blob/main/Pr2_ProcesamientoVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install opencv_jupyter_ui
!pip install -q ipycanvas==0.11
from google.colab import output
output.enable_custom_widget_manager()
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.7/255.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.3 MB/s eta 0:00:00


In [2]:
import cv2
from matplotlib import pyplot as plt
import opencv_jupyter_ui as jcv2

In [3]:
from google.colab import files
uploadfile=files.upload()

Saving Mall_Security_Cam_edited.mp4 to Mall_Security_Cam_edited.mp4


In [ ]:
# This part does not work because of cv2.imshow, that does not work with Google Colab

cap = cv2.VideoCapture("Mall_Security_Cam.mp4")
Backremover = cv2.createBackgroundSubtractorMOG2() #que me quita el fondo, y se ejecuta en la stack de servicio junto a la CPU
#si quiero aplicarlo a una foto, linea ****
while(cap.isOpened):
# ahora tengo que ver la imagen que me interesa
  ret, frame = cap.read() #me da primero un vlaor de retornno, si ha podido conseguir la imagen, y después la imagen
  # si el fichero se corrompe y el string se pierde, me marca como false, eso es que entonces que no lo procesa, para evitar que pete
  #puedo poner _ para que no me lo guarde, pero lo que no puedo hacer es no leerlo

  people = Backremover.apply(frame);
  # Finding nearby pixels to be considered as a person
  clusters, _= cv2.findContours(people, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

   for cnt in clusters:
    area = cv2.contourArea(cnt)
    if area < 20:
      # Drawing the contours with green color
      cv2.drawContours(frame, [cnt], -1, (0,255,0),2)
      # Draws a rectangle in every centroid and saves the coordinates
      x,y,w,h = cv2.boundingRect(cnt)
  # It should show the video frame by frame
  cv2.imshow("video",frame);

  # To be aware of problems of chrashing
  if cv2.waitKey(1) == ord('q'):
    break
  cap.release()
  cv2.destroyAllWindows()

In [ ]:
from google.colab.patches import cv2_imshow
# Video
cap = cv2.VideoCapture("Mall_Security_Cam_edited.mp4")

# Check if it was opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Reading of the first two frames (in order)
ret, frame1 = cap.read(1)
ret, frame2 = cap.read(2)

while(cap.isOpened()):
  # Absolute difference (distance) from consecutive frames
  diff = cv2.absdiff(frame1,frame2)
  # Conversion to gray scale
  gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
  # Filter blur function
  blur = cv2.GaussianBlur(gray, (5,5), 0)
  # Mask to keep just the objects inside the threshold choosen
  _, mask = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
  # Dilate the created mask
  dil = cv2.dilate(mask, None, iterations=5)
  # Getting the different contours
  cont, _ = cv2.findContours(dil, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  for cnt in cont:
    (x, y, w, h) = cv2.boundingRect(cnt)

    if cv2.contourArea(cnt) < 450:
      continue
    cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(frame1, "People Moving", (10, 20), cv2.FONT_HERSHEY_TRIPLEX,
                1, (255, 0, 0), 3)

  # Frame showing
  jcv2.imshow('feed', frame1)
  # Getting/reading the next frames
  frame1 = frame2
  ret, frame2 = cap.read()

  if ret is not True:
    break

  if cv2.waitKey(1) == ord('q'):
    break

cv2.destroyAllWindows()
cap.release()